# Zeiler & Fergus Architecture

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import os

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
train_path = os.path.join('..', '..', 'data', 'data_subset', 'data_train')
test_path = os.path.join('..', '..', 'data', 'data_subset', 'data_test')

In [ ]:
# Hyperparameters
train_batch_size = 32
test_batch_size = 32

In [ ]:
# QUESTION: Do we need to augment? Not augmenting for now. Loading in batches to fit in memory.
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(
    train_path,
    target_size=(220, 220),
    batch_size=train_batch_size,
    # class_mode= # What to include here?
)

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(
    test_path,
    target_size=(220, 220),
    batch_size=train_batch_size,
    # class_mode= # What to include here?
)

In [ ]:
# More hyperparameters
learning_rate = 1e-5
epochs = 100

# Math works out so that one epoch equals one pass through the dataset.
train_steps_per_epoch = train_datagen.n // train_batch_size
test_steps_per_epoch = test_datagen.n // test_batch_size

In [ ]:
# Callback for early stopping
es_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=5,
    restore_best_weights=True
)

In [2]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', input_shape=(220, 220, 3), name='conv1'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool1'),
    # Assuming rnorm means batch normalization.
    keras.layers.BatchNormalization(name='rnorm1'),
    
    keras.layers.Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='conv2a'),
    keras.layers.Conv2D(192, (3, 3), strides=(1, 1), padding='same', name='conv2'),
    keras.layers.BatchNormalization(name='rnorm2'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool2'),
    
    keras.layers.Conv2D(192, (1, 1), strides=(1, 1), padding='same', name='conv3a'),
    keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', name='conv3'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool3'),
    
    keras.layers.Conv2D(384, (1, 1), strides=(1, 1), padding='same', name='conv4a'),
    keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', name='conv4'),
    
    keras.layers.Conv2D(256, (1, 1), strides=(1, 1), padding='same', name='conv5a'),
    keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', name='conv5'),
    
    keras.layers.Conv2D(256, (1, 1), strides=(1, 1), padding='same', name='conv6a'),
    keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', name='conv6'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same', name='pool4'),
    
    # ---------- Questions about this section. Discuss next meet. ----------
    keras.layers.Flatten(),
], name='zeiler_fergus')

In [3]:
model.summary()

Model: "zeiler_fergus"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 110, 110, 64)      9472      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 55, 55, 64)        0         
_________________________________________________________________
rnorm1 (BatchNormalization)  (None, 55, 55, 64)        256       
_________________________________________________________________
conv2a (Conv2D)              (None, 55, 55, 64)        4160      
_________________________________________________________________
conv2 (Conv2D)               (None, 55, 55, 192)       110784    
_________________________________________________________________
rnorm2 (BatchNormalization)  (None, 55, 55, 192)       768       
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 28, 28, 192)     